Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---


---------
Problem 1
---
Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor t using nn.l2_loss(t). The right amount of regularization should improve your validation / test accuracy.


In [5]:
train_subset = 10000
beta = 0.01

In [6]:
graph = tf.Graph()

In [7]:
with graph.as_default():
    
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
  
    logits = tf.matmul(tf_train_dataset, weights) + biases 
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels,logits=logits))
    
    regularizer = tf.nn.l2_loss(weights)
    loss = tf.reduce_mean(loss + beta * regularizer)
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax( tf.matmul(tf_valid_dataset, weights) + biases )
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [8]:
num_steps = 901

In [9]:
with tf.Session(graph=graph) as session:
    # This is a one-time operation which ensures the parameters get initialized as
    # we described in the graph: random weights for the matrix, zeros for the
    # biases. 
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
        _, l, predictions = session.run([optimizer, loss, train_prediction])
        if (step % 100 == 0):
            print('Loss at step {}: {}'.format(step, l))
            print('Training accuracy: {:.1f}'.format(accuracy(predictions, 
                                                         train_labels[:train_subset, :])))
            # Calling .eval() on valid_prediction is basically like calling run(), but
            # just to get that one numpy array. Note that it recomputes all its graph
            # dependencies.
            
            # You don't have to do .eval above because we already ran the session for the
            # train_prediction
            print('Validation accuracy: {:.1f}'.format(accuracy(valid_prediction.eval(), 
                                                           valid_labels)))
    print('Test accuracy: {:.1f}'.format(accuracy(test_prediction.eval(), test_labels))) 

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Loss at step 0: 49.5975227355957
Training accuracy: 7.9
Validation accuracy: 10.5
Loss at step 100: 11.793355941772461
Training accuracy: 73.2
Validation accuracy: 71.4
Loss at step 200: 4.478046417236328
Training accuracy: 78.6
Validation accuracy: 76.4
Loss at step 300: 1.989488124847412
Training accuracy: 82.0
Validation accuracy: 79.4
Loss at step 400: 1.1425964832305908
Training accuracy: 83.4
Validation accuracy: 80.9
Loss at step 500: 0.851237416267395
Training accuracy: 83.8
Validation accuracy: 81.3
Loss at step 600: 0.7498509883880615
Training accuracy: 84.1
Validation accuracy: 81.5
Loss at step 700: 0.7142601013183594
Training accuracy: 84.1
Validation accuracy: 81.6
Loss at step 800: 0.7016791105270386
Training accuracy: 84.2
Validation accuracy: 81.7
Loss at step 900: 0.6972043514251709
Training accuracy: 84.3
Validation accuracy: 81.6
Test accuracy: 88.9


---
with Relu and Hidden Layer
---------
---

In [10]:
num_hidden_size = 1024
batch_size = 128
beta

0.01

In [11]:
with graph.as_default():
    
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden_size]))
    biases_1 = tf.Variable(tf.zeros([num_hidden_size]))
    weights_2 = tf.Variable(tf.truncated_normal([num_hidden_size, num_labels]))
    biases_2 = tf.Variable(tf.zeros([num_labels]))
  
    logits = tf.matmul(
             tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1), weights_2) + biases_2
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels,logits=logits))
    
    regularizer = tf.nn.l2_loss(weights)
    loss = tf.reduce_mean(loss + beta * regularizer)
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1), weights_2) + biases_2)
    test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1), weights_2) + biases_2)

In [12]:
num_steps = 3001

In [13]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {
            tf_train_dataset : batch_data,
            tf_train_labels : batch_labels
        }
        
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Minibatch loss at step {}: {}".format(step, l))
            print("Minibatch accuracy: {:.1f}".format(accuracy(predictions, batch_labels)))
            print("Validation accuracy: {:.1f}".format(accuracy(valid_prediction.eval(), valid_labels)))
    print("Test accuracy: {:.1f}".format(accuracy(test_prediction.eval(), test_labels)))

Initialized
Minibatch loss at step 0: 340.1021728515625
Minibatch accuracy: 14.8
Validation accuracy: 23.2
Minibatch loss at step 500: 25.479236602783203
Minibatch accuracy: 71.1
Validation accuracy: 80.4
Minibatch loss at step 1000: 18.312572479248047
Minibatch accuracy: 80.5
Validation accuracy: 80.6
Minibatch loss at step 1500: 12.253643989562988
Minibatch accuracy: 78.9
Validation accuracy: 79.1
Minibatch loss at step 2000: 9.486692428588867
Minibatch accuracy: 75.8
Validation accuracy: 80.3
Minibatch loss at step 2500: 3.4978082180023193
Minibatch accuracy: 78.9
Validation accuracy: 81.2
Minibatch loss at step 3000: 9.046502113342285
Minibatch accuracy: 81.2
Validation accuracy: 81.4
Test accuracy: 88.6


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [14]:
num_steps = 3001

train_dataset_2 = train_dataset[:500,:]
train_labels_2 = train_labels[:500]

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        
        offset = (step * batch_size) % (train_labels_2.shape[0] - batch_size)
        
        batch_data = train_dataset_2[offset:(offset + batch_size), :]
        batch_labels = train_labels_2[offset:(offset + batch_size), :]
        
        feed_dict = {
            tf_train_dataset : batch_data,
            tf_train_labels : batch_labels
        }
        
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Minibatch loss at step {}: {}".format(step, l))
            print("Minibatch accuracy: {:.1f}".format(accuracy(predictions, batch_labels)))
            print("Validation accuracy: {:.1f}".format(accuracy(valid_prediction.eval(), valid_labels)))
    print("Test accuracy: {:.1f}".format(accuracy(test_prediction.eval(), test_labels)))

Initialized
Minibatch loss at step 0: 350.6313781738281
Minibatch accuracy: 10.2
Validation accuracy: 31.6
Minibatch loss at step 500: 0.2855239510536194
Minibatch accuracy: 99.2
Validation accuracy: 74.9
Minibatch loss at step 1000: 0.6987132430076599
Minibatch accuracy: 99.2
Validation accuracy: 75.0
Minibatch loss at step 1500: 0.3522646129131317
Minibatch accuracy: 99.2
Validation accuracy: 75.0
Minibatch loss at step 2000: 0.05342712253332138
Minibatch accuracy: 99.2
Validation accuracy: 75.0
Minibatch loss at step 2500: 0.07131868600845337
Minibatch accuracy: 99.2
Validation accuracy: 75.0
Minibatch loss at step 3000: 0.0001278442214243114
Minibatch accuracy: 100.0
Validation accuracy: 75.0
Test accuracy: 83.2


---
Problem 3
----------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [15]:
num_hidden_size

1024

In [16]:
with graph.as_default():
    
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden_size]))
    biases_1 = tf.Variable(tf.zeros([num_hidden_size]))
    
    weights_2 = tf.Variable(tf.truncated_normal([num_hidden_size, num_labels]))
    biases_2 = tf.Variable(tf.zeros([num_labels]))
  
    logits = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)
    keep_prob = tf.placeholder("float")
    logits = tf.nn.dropout(logits,keep_prob)
    logits = tf.matmul(logits,weights_2) + biases_2
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels,logits=logits))
    
    regularizers = tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2)
    loss = tf.reduce_mean(loss + beta * regularizer)
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    
    logits = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1)
#     keep_prob = tf.placeholder("float")
#     logits = tf.nn.dropout(logits,keep_prob)
    logits = tf.matmul(logits,weights_2) + biases_2
    
    valid_prediction = tf.nn.softmax(logits)
    
    logits = tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1)
#     keep_prob = tf.placeholder("float")
#     logits = tf.nn.dropout(logits,keep_prob)
    logits = tf.matmul(logits,weights_2) + biases_2
    
    test_prediction = tf.nn.softmax(logits)

In [17]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {
            tf_train_dataset : batch_data,
            tf_train_labels : batch_labels,
            keep_prob : 0.5
        }
        
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Minibatch loss at step {}: {}".format(step, l))
            print("Minibatch accuracy: {:.1f}".format(accuracy(predictions, batch_labels)))
            print("Validation accuracy: {:.1f}".format(accuracy(valid_prediction.eval(), valid_labels)))
    print("Test accuracy: {:.1f}".format(accuracy(test_prediction.eval(), test_labels)))

Initialized
Minibatch loss at step 0: 513.34619140625
Minibatch accuracy: 9.4
Validation accuracy: 19.9
Minibatch loss at step 500: 36.724430084228516
Minibatch accuracy: 66.4
Validation accuracy: 79.3
Minibatch loss at step 1000: 24.718204498291016
Minibatch accuracy: 71.9
Validation accuracy: 80.1
Minibatch loss at step 1500: 13.10416316986084
Minibatch accuracy: 75.0
Validation accuracy: 79.8
Minibatch loss at step 2000: 19.1185245513916
Minibatch accuracy: 78.1
Validation accuracy: 77.8
Minibatch loss at step 2500: 7.437963485717773
Minibatch accuracy: 72.7
Validation accuracy: 79.8
Minibatch loss at step 3000: 10.99866771697998
Minibatch accuracy: 72.7
Validation accuracy: 79.5
Test accuracy: 87.6


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [18]:
import math as math

In [30]:
batch_size = 256
beta = 0.001

hidden_nodes_1 = 1024
hidden_nodes_2 = int(hidden_nodes_1 * 0.5)
hidden_nodes_3 = int(hidden_nodes_1 * np.power(0.5, 2))
hidden_nodes_4 = int(hidden_nodes_1 * np.power(0.5, 3))
hidden_nodes_5 = int(hidden_nodes_1 * np.power(0.5, 4))

In [31]:
graph = tf.Graph()
with graph.as_default():
    
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes_1], stddev=math.sqrt(2.0/(image_size*image_size))))
    biases_1 = tf.Variable(tf.zeros([hidden_nodes_1]))
    
    weights_2 = tf.Variable(tf.truncated_normal([hidden_nodes_1, hidden_nodes_2], stddev=math.sqrt(2.0/hidden_nodes_1)))
    biases_2 = tf.Variable(tf.zeros([hidden_nodes_2]))
    
    weights_3 = tf.Variable(tf.truncated_normal([hidden_nodes_2, hidden_nodes_3], stddev=math.sqrt(2.0/hidden_nodes_2)))
    biases_3 = tf.Variable(tf.zeros([hidden_nodes_3]))
    
    weights_4 = tf.Variable(tf.truncated_normal([hidden_nodes_3, hidden_nodes_4], stddev=math.sqrt(2.0/hidden_nodes_3)))
    biases_4 = tf.Variable(tf.zeros([hidden_nodes_4]))
    
    weights_5 = tf.Variable(tf.truncated_normal([hidden_nodes_4, hidden_nodes_5], stddev=math.sqrt(2.0/hidden_nodes_4)))
    biases_5 = tf.Variable(tf.zeros([hidden_nodes_5]))
    
    weights_6 = tf.Variable(tf.truncated_normal([hidden_nodes_5, num_labels], stddev=math.sqrt(2.0/hidden_nodes_5)))
    biases_6 = tf.Variable(tf.zeros([num_labels]))
    
    logit = tf.matmul(tf_train_dataset, weights_1) + biases_1
    logit = tf.nn.relu(logit)
    keep_prob = tf.placeholder("float")
    logit = tf.nn.dropout(logit, keep_prob)
    
    logit = tf.matmul(logit, weights_2) + biases_2
    logit = tf.nn.relu(logit)
#     keep_prob = tf.placeholder("float")
    logit = tf.nn.dropout(logit, keep_prob)
    
    logit = tf.matmul(logit, weights_3) + biases_3
    logit = tf.nn.relu(logit)
#     keep_prob = tf.placeholder("float")
    logit = tf.nn.dropout(logit, keep_prob)
    
    logit = tf.matmul(logit, weights_4) + biases_4
    logit = tf.nn.relu(logit)
#     keep_prob = tf.placeholder("float")
    logit = tf.nn.dropout(logit, keep_prob)
    
    logit = tf.matmul(logit, weights_5) + biases_5
    logit = tf.nn.relu(logit)
#     keep_prob = tf.placeholder("float")
    logit = tf.nn.dropout(logit, keep_prob)
    
    logit = tf.matmul(logit, weights_6) + biases_6
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logit, labels = tf_train_labels))
    
    regularizers = tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2) + \
                   tf.nn.l2_loss(weights_3) + tf.nn.l2_loss(weights_4) + \
                   tf.nn.l2_loss(weights_5) + tf.nn.l2_loss(weights_6)
    loss = tf.reduce_mean(loss + beta * regularizers)
    
    global_step = tf.Variable(0)
    start_learning_rate = 0.5
    learning_rate = tf.train.exponential_decay(start_learning_rate, global_step, 100000, 0.96, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    train_prediction = tf.nn.softmax(logit)
    
    logit = tf.matmul(tf_valid_dataset, weights_1) + biases_1
    logit = tf.nn.relu(logit)
    
    logit = tf.matmul(logit, weights_2) + biases_2
    logit = tf.nn.relu(logit)
    
    logit = tf.matmul(logit, weights_3) + biases_3
    logit = tf.nn.relu(logit)
    
    logit = tf.matmul(logit, weights_4) + biases_4
    logit = tf.nn.relu(logit)
    
    logit = tf.matmul(logit, weights_5) + biases_5
    logit = tf.nn.relu(logit)
    
    logit = tf.matmul(logit,weights_6) + biases_6
    
    valid_prediction = tf.nn.softmax(logit)
    
    logit = tf.matmul(tf_test_dataset, weights_1) + biases_1
    logit = tf.nn.relu(logit)
    
    logit = tf.matmul(logit, weights_2) + biases_2
    logit = tf.nn.relu(logit)
    
    logit = tf.matmul(logit, weights_3) + biases_3
    logit = tf.nn.relu(logit)
    
    logit = tf.matmul(logit, weights_4) + biases_4
    logit = tf.nn.relu(logit)
    
    logit = tf.matmul(logit, weights_5) + biases_5
    logit = tf.nn.relu(logit)
    
    logit = tf.matmul(logit,weights_6) + biases_6
    
    test_prediction = tf.nn.softmax(logit)
    
    

In [32]:
num_steps = 15000

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]

        feed_dict = {
            tf_train_dataset : batch_data,
            tf_train_labels : batch_labels,
            keep_prob : 0.5
        }
        
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)

        if (step % 500 == 0):
                print("Minibatch loss at step {}: {}".format(step, l))
                print("Minibatch accuracy: {:.1f}".format(accuracy(predictions, batch_labels)))
                print("Validation accuracy: {:.1f}".format(accuracy(valid_prediction.eval(), valid_labels)))
    print("Test accuracy: {:.1f}".format(accuracy(test_prediction.eval(), test_labels)))

Initialized
Minibatch loss at step 0: 4.964610576629639
Minibatch accuracy: 9.4
Validation accuracy: 10.4
Minibatch loss at step 500: 1.6070706844329834
Minibatch accuracy: 83.2
Validation accuracy: 83.8
Minibatch loss at step 1000: 1.2926691770553589
Minibatch accuracy: 83.2
Validation accuracy: 85.0
Minibatch loss at step 1500: 0.9492639303207397
Minibatch accuracy: 87.5
Validation accuracy: 85.7
Minibatch loss at step 2000: 0.9010024070739746
Minibatch accuracy: 84.0
Validation accuracy: 86.5
Minibatch loss at step 2500: 0.8876545429229736
Minibatch accuracy: 82.4
Validation accuracy: 85.7
Minibatch loss at step 3000: 0.8265504837036133
Minibatch accuracy: 82.0
Validation accuracy: 86.8
Minibatch loss at step 3500: 0.6111668944358826
Minibatch accuracy: 89.8
Validation accuracy: 86.8
Minibatch loss at step 4000: 0.6868302226066589
Minibatch accuracy: 83.6
Validation accuracy: 87.0
Minibatch loss at step 4500: 0.6629272103309631
Minibatch accuracy: 85.2
Validation accuracy: 87.1
Mini